In [60]:
import serial
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import time

In [61]:
ser = serial.Serial('COM6', 115200, timeout=1)

In [66]:
start = False
data_buffer = []

# start time
start_time = time.time()

while True:
    if ser.in_waiting > 0:
        received_data = ser.readline()

        # check if 10 s has passed
        time_end = time.time()
        deltatime = time_end - start_time
        if deltatime < 5:
            continue
        
        # Check for start and end markers
        if received_data == b'fffffff1\r\n':
            print(f"Start")
            start = True
            time_start = time.time()
            continue
        elif received_data == b'f1ffffff\r\n':
            print(f"Saved @ {time_end}")
            time_start = time.time()
            start = False
        if start:
            data_buffer = received_data.decode('utf-8').replace('\r\n', '').split(';')
            if len(data_buffer) == 768:
                data_list = list(map(float, data_buffer))
            
            # # Filter out zero values and reshape to a 2D array (24x32 for MLX90640 sensor)
            data_array = np.array(data_list).reshape((24, 32)) + 8

            # # save data array to json to ../data/data_output/
            # with open(f'../data/output_data/thermal_image_{time.strftime("%y%d%m-%H%M%S")}.json', 'w') as f:
            #     # write current timestamp
            #     f.write(time.strftime("%y%d%m-%H%M%S"))
            #     f.write('\n')
            #     # write data array
            #     f.write(str(data_array.tolist()))
            
            # Plot the thermal image
            
            # plt.imshow(data_array, cmap='jet', interpolation='nearest')
            # plt.colorbar(label='Temperature (°C)')
            # plt.title('Thermal Image')
            # plt.show()
			

Start
Saved @ 1718104671.9847114
Start
Saved @ 1718104673.6401489
Start
Saved @ 1718104675.2960691
Start
Saved @ 1718104676.9528723
Start
Saved @ 1718104678.610281


KeyboardInterrupt: 

In [ ]:
ser.close()

In [ ]:
start = False

# Set up the initial plot
plt.ion()  # Turn on interactive mode
fig, ax = plt.subplots()
thermal_image = ax.imshow(np.zeros((24, 32)), cmap='jet', interpolation='nearest')
cbar = plt.colorbar(thermal_image, ax=ax, label='Temperature (°C)')
plt.show()

while True:
    if ser.in_waiting > 0:
        received_data = ser.readline()
        
        # Check for start and end markers
        if received_data == b'fffffff1\r\n':
            print(f"Received data: {received_data} (start)")
            start = True
            data_buffer = []  # Reset the data buffer at the start
            continue
        elif received_data == b'f1ffffff\r\n':
            print(f"Received data: {received_data} (end)")
            start = False
            if len(data_buffer) == 768:
                data_list = list(map(float, data_buffer))
                
                # Filter out zero values and reshape to a 2D array (24x32 for MLX90640 sensor)
                data_array = np.array(data_list).reshape((24, 32)) + 8

                # Apply Gaussian filter with a 5x5 kernel
                data_array = gaussian_filter(data_array, sigma=1)

                # Update the thermal image data
                thermal_image.set_data(data_array)
                thermal_image.set_clim(vmin=np.min(data_array), vmax=np.max(data_array))
                
                # Redraw the plot
                plt.draw()
                plt.pause(0.1)  # Pause to allow the plot to update
            else:
                print("Data buffer length is incorrect. Skipping update.")
        elif start:
            # Append received data to buffer
            data_buffer.extend(received_data.decode('utf-8').replace('\r\n', '').split(';'))


In [ ]:
ser.close()

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import time

# Create a figure and axis
fig, ax = plt.subplots()
xdata, ydata = [], []
line, = ax.plot([], [], 'r-')

# Initialize the plot
ax.set_xlim(0, 2 * np.pi)
ax.set_ylim(-1, 1)

# Draw the initial plot
plt.ion()
plt.show()

# Loop to update the plot
for frame in np.linspace(0, 2 * np.pi, 128):
    xdata.append(frame)
    ydata.append(np.sin(frame))
    line.set_data(xdata, ydata)
    ax.draw_artist(ax.patch)
    ax.draw_artist(line)
    fig.canvas.blit(ax.bbox)
    fig.canvas.flush_events()
    plt.pause(0.1)  # Pause to update the plot

plt.ioff()
plt.show()
